In [ ]:
# Import QRCode from pyqrcode
import pyqrcode
import png
from pyqrcode import QRCode


s = "1 veg_curry B"
  
# Generate QR code
url = pyqrcode.create(s)
  
# Create and save the png file naming "myqr.png"
url.png('veg.png', scale = 8)

In [ ]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode


cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray_img = cv2.cvtColor(frame,0)
    barcode = decode(gray_img)

    for obj in barcode:
        points = obj.polygon
        (x,y,w,h) = obj.rect
        pts = np.array(points, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(frame, [pts], True, (0, 255, 0), 3)

        barcodeData = obj.data.decode("utf-8")
        barcodeType = obj.type
        string = "Data " + str(barcodeData) + " | Type " + str(barcodeType)
        
        cv2.putText(frame, string, (x,y), cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0), 2)
        #print("Barcode: "+barcodeData +" | Type: "+barcodeType)
        
    
    cv2.imshow('Image', frame)
    if cv2.waitKey(1)==13:
        break
print("Barcode: "+barcodeData)
cap.release()
cv2.destroyAllWindows()

In [ ]:
import smtplib

print(barcodeData)

def Convert(string):
    li = list(string.split(" "))
    return li
  
# Driver code    
str1 =barcodeData
list1 = Convert(str1)
list1
st1= list1[0]
sender = "knownoutlander@gmail.com"
receiver = "rajzamal123@gmail.com"
message = f"""\
Subject: Hi Admin
To: {receiver}
From: {sender}
Item number {st1} is category C deliver it fast."""

server = smtplib.SMTP_SSL("smtp.gmail.com",465)
server.login("knownoutlander@gmail.com","Test@7250#")
if(list1[2] == 'B'):
    server.sendmail("knownoutlander@gmail.com","rajzamal123@gmail.com",message)
    server.quit()
    

In [ ]:
# Module 2 - Create a Cryptocurrency

# To be installed:
# Flask==0.12.2: pip install Flask==0.12.2
# Postman HTTP Client: https://www.getpostman.com/
# requests==2.18.4: pip install requests==2.18.4

# Importing the libraries
import datetime
import hashlib
import json
from flask import Flask, jsonify, request
import requests
from uuid import uuid4
from urllib.parse import urlparse

# Part 1 - Building a Blockchain

class Blockchain:

    def __init__(self):
        self.chain = []
        self.transactions = []
        self.create_block(proof = 1, previous_hash = '0')
        self.nodes = set()
    
    def create_block(self, proof, previous_hash):
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash,
                 'transactions': self.transactions}
        self.transactions = []
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof
    
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
            previous_block = block
            block_index += 1
        return True
    
    def add_transaction(self, item_no, item_name , item_quality):
        self.transactions.append({'item_no': item_no,
                                  'item_name': item_name,
                                  'item_quality': item_quality})
        previous_block = self.get_previous_block()
        return previous_block['index'] + 1
    
    def add_node(self, address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)
    
    def replace_chain(self):
        network = self.nodes
        longest_chain = None
        max_length = len(self.chain)
        for node in network:
            response = requests.get(f'http://{node}/get_chain')
            if response.status_code == 200:
                length = response.json()['length']
                chain = response.json()['chain']
                if length > max_length and self.is_chain_valid(chain):
                    max_length = length
                    longest_chain = chain
        if longest_chain:
            self.chain = longest_chain
            return True
        
        return False
    
    
    def concensus(self,item_no, item_name, item_quality):
        network = self.nodes
        flag=0
        max_length = len(self.chain)
        for node in network:
            response = requests.post(f'http://{node}/concensus', json={'item_no': item_no,
                                  'item_name': item_name,
                                  'item_quality': item_quality})
            if response.status_code == 200:
                flag=flag+1
        if flag > (max_length/2):
            return True


# Part 2 - Mining our Blockchain

# Creating a Web App
app = Flask(__name__)

# Creating an address for the node on Port 5001
node_address = str(uuid4()).replace('-', '')

# Creating a Blockchain
blockchain = Blockchain()

# Mining a new block
@app.route('/mine_block', methods = ['GET'])
def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    blockchain.add_transaction(item_no = 0, item_name ='IOT_1', item_quality = 'A')
    block = blockchain.create_block(proof, previous_hash)
    response = {'message': 'Congratulations, you just mined a block!',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash'],
                'transactions': block['transactions']}
    return jsonify(response), 200

# Getting the full Blockchain
@app.route('/get_chain', methods = ['GET'])
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return jsonify(response), 200

# Checking if the Blockchain is valid
@app.route('/is_valid', methods = ['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message': 'All good. The Blockchain is valid.'}
    else:
        response = {'message': 'Houston, we have a problem. The Blockchain is not valid.'}
    return jsonify(response), 200

# Adding a new transaction to the Blockchai n
@app.route('/add_transaction', methods = ['POST'])
def add_transaction():
    json = request.get_json()
    transaction_keys = ['item_no', 'item_name', 'item_quality'] 
    #if type(transaction_keys[0]) == int and type(transaction_keys[1]) == str and type(transaction_keys[2]) == str:
        #count+=1
        #print(count)
    #else:
        #return 'Invalid Input', 400
    
    if not all(key in json for key in transaction_keys):
        return 'Some elements of the transaction are missing', 400
    index = blockchain.add_transaction(json['item_no'], json['item_name'], json['item_quality'])
    is_input_valid = blockchain.concensus(json['item_no'], json['item_name'], json['item_quality'])
    if is_input_valid:
        response = {'message': f'Parameters are valid & this transaction will be added to Block {index}'}
        return jsonify(response), 201
    else:
        return jsonify(response), 40
# Part 3 - Decentralizing our Blockchain

# Connecting new nodes
@app.route('/connect_node', methods = ['POST'])
def connect_node():
    json = request.get_json()
    nodes = json.get('nodes')
    if nodes is None:
        return "No node", 400
    for node in nodes:
        blockchain.add_node(node)
    response = {'message': 'All the nodes are now connected. The Hadcoin Blockchain now contains the following nodes:',
                'total_nodes': list(blockchain.nodes)}
    return jsonify(response), 201

# Replacing the chain by the longest chain if needed
@app.route('/replace_chain', methods = ['GET'])
def replace_chain():
    is_chain_replaced = blockchain.replace_chain()
    if is_chain_replaced:
        response = {'message': 'The nodes had different chains so the chain was replaced by the longest one.',
                    'new_chain': blockchain.chain}
    else:
        response = {'message': 'All good. The chain is the largest one.',
                    'actual_chain': blockchain.chain}
    return jsonify(response), 200

@app.route('/concensus', methods = ['POST'])
def concensus():
    json = request.get_json()
    response = {'message': f'json'}

    if type(json['item_no']) == int and type(json['item_name']) == str and type(json['item_quality']) == str:
        return jsonify(response), 200
    else:
        return jsonify(response), 400


# Running the app
app.run(host = '0.0.0.0', port = 5001)

In [25]:
import requests


urlconnect1 = requests.post("http://127.0.0.1:5001/connect_node",json={
    "nodes": ["http://127.0.0.1:5002"]
})
urlconnect1 = requests.post("http://127.0.0.1:5002/connect_node",json={
    "nodes": ["http://127.0.0.1:5001"]
})
urlget = requests.get("http://127.0.0.1:5001/get_chain")
print(urlget.json())

{'chain': [{'index': 1, 'previous_hash': '0', 'proof': 1, 'timestamp': '2021-08-30 21:16:32.061709', 'transactions': []}], 'length': 1}


In [9]:
import cv2
import smtplib
import numpy as np
from pyzbar.pyzbar import decode


cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray_img = cv2.cvtColor(frame,0)
    barcode = decode(gray_img)

    for obj in barcode:
        points = obj.polygon
        (x,y,w,h) = obj.rect
        pts = np.array(points, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(frame, [pts], True, (0, 255, 0), 3)

        barcodeData = obj.data.decode("utf-8")
        barcodeType = obj.type
        string = "Data " + str(barcodeData) + " | Type " + str(barcodeType)
        
        cv2.putText(frame, string, (x,y), cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0), 2)
        #print("Barcode: "+barcodeData +" | Type: "+barcodeType)
        
    
    cv2.imshow('Image', frame)
    if cv2.waitKey(1)==13:
        break
print("Barcode: "+barcodeData)
cap.release()
cv2.destroyAllWindows()


print(barcodeData)

def Convert(string):
    li = list(string.split(" "))
    return li
  
# Driver code    
str1 =barcodeData
list1 = Convert(str1)
list1
st1= list1[0]
sender = "knownoutlander@gmail.com"
receiver = "rajzamal123@gmail.com"
message = f"""\
Subject: Hi Admin
To: {receiver}
From: {sender}
Item number {st1} is category C deliver it fast."""

server = smtplib.SMTP_SSL("smtp.gmail.com",465)
server.login("knownoutlander@gmail.com","Test@7250#")
if(list1[2] == 'C'):
    server.sendmail("knownoutlander@gmail.com","rajzamal123@gmail.com",message)
    server.quit()



Barcode: 1 veg_curry B
1 veg_curry B


In [17]:
list1[0]

'1'

In [30]:
#import requests
#num =int(list1[0])
#print(num)

#url2 = requests.post("http://127.0.0.1:5001/add_transaction",json={'item_no': num,
#                                  'item_name': list1[1],
#                                  'item_quality': list1[2]})

url2 = requests.post("http://127.0.0.1:5001/add_transaction",json={'item_no': 57,
                                  'item_name': 'food',
                                  'item_quality': 'B'})
requests.get("http://127.0.0.1:5001/mine_block")
url = requests.get("http://127.0.0.1:5001/get_chain")
print(url.json())
print(url2)
 

{'chain': [{'index': 1, 'previous_hash': '0', 'proof': 1, 'timestamp': '2021-08-30 21:16:32.061709', 'transactions': []}, {'index': 2, 'previous_hash': 'c13fa900faa7db4fd2d087ec407b0d86e1eb8d35452e23de84ae1c88a7eff6fb', 'proof': 533, 'timestamp': '2021-08-30 21:17:14.609256', 'transactions': [{'item_name': 'fszfs', 'item_no': 57, 'item_quality': 'B'}, {'item_name': 'IOT_1', 'item_no': 0, 'item_quality': 'A'}]}, {'index': 3, 'previous_hash': '188a6943ccd57762e75844959bea6ca857ac34378c1508e89690fe6a4ecb0af7', 'proof': 45293, 'timestamp': '2021-08-30 21:17:44.402793', 'transactions': [{'item_name': 'IOT_1', 'item_no': 0, 'item_quality': 'A'}]}, {'index': 4, 'previous_hash': 'bba08cf48f974d654cbe5cb434c0c8f4d1432d4ecd3a2c7eee46187604948056', 'proof': 21391, 'timestamp': '2021-08-30 21:18:05.042248', 'transactions': [{'item_name': 'IOT_1', 'item_no': 0, 'item_quality': 'A'}]}, {'index': 5, 'previous_hash': '0c5320e773f8c0f15851c4987492f4e225feb10b00f006764c898b04cd8d6281', 'proof': 8018, 't

In [3]:
requests.get("http://127.0.0.1:5001/mine_block")
url = requests.get("http://127.0.0.1:5001/get_chain")
print(url.json())


{'chain': [{'index': 1, 'previous_hash': '0', 'proof': 1, 'timestamp': '2021-08-30 20:28:59.550429', 'transactions': []}, {'index': 2, 'previous_hash': '49624adfec0919d78735922d9b6ff62e734aed18b21a0d6a00173ee0b7dd7a7c', 'proof': 533, 'timestamp': '2021-08-30 20:29:48.205137', 'transactions': [{'item_name': 'IOT_1', 'item_no': 0, 'item_quality': 'A'}]}, {'index': 3, 'previous_hash': 'fd9218ab385860a7d17ab74a72326db2c7eb8dd900a4fc2afe967d7502187b49', 'proof': 45293, 'timestamp': '2021-08-30 20:30:04.599480', 'transactions': [{'item_name': 'IOT_1', 'item_no': 0, 'item_quality': 'A'}]}], 'length': 3}


CONSENSUS

In [ ]:

lst24 = [1,'apple','A']
type(lst24[0])
count = 0
if type(lst24[0]) == int and type(lst24[1]) == str and type(lst24[2]) == str:
    count+=1
    print(count)
else:
    print(2)

In [ ]:
import requests
url = 'https://www.w3schools.com/python/demopage.php'
myobj = {'somekey': 'somevalue'}

x = requests.post(url, data = myobj)

print(x)